In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import re

In [2]:
driver = webdriver.Chrome()
driver.get('https://airtable.com/app1PaujS9zxVGUZ4/shrqYt5kSqMzHV9R5/tbl8c8kanuNB6bPYr?backgroundColor=green&viewControls=on',)
time.sleep(1)

In [3]:
scrollbar = "//*[@id= 'view']/div/div[1]/div[1]/div[3]"
element = driver.find_element(By.XPATH, scrollbar)


In [4]:
import time

leftPane = []
rightPane = []

i = 0
while i < 120:
    start_time = time.time()  # Start time of the loop

    ActionChains(driver).click_and_hold(element).move_by_offset(0,1).perform()
    ActionChains(driver).reset_actions()

    html = driver.page_source
    parsed_html = BeautifulSoup(html, "html.parser")
    leftPane.append(parsed_html.find_all('div', class_='dataRow leftPane rowExpansionEnabled rowSelectionEnabled'))
    rightPane.append(parsed_html.find_all('div', class_='dataRow rightPane rowExpansionEnabled rowSelectionEnabled'))
    
    i += 1

    end_time = time.time()  # End time of the loop
    elapsed_time = end_time - start_time  # Time taken for one loop iteration

    # If the operation is faster than 1/5 second, pause for the remaining time to comply with the rate limit
    if elapsed_time < 1/5:
        time.sleep(1/5 - elapsed_time)


In [5]:
dictionary = {}
for i in range(len(leftPane)):
    for j in range(len(leftPane[i])):
        pattern = r'<div class="line-height-4 overflow-hidden truncate-block-1-line truncate">(.*?)<\/div>'
        match = re.search(pattern, str(leftPane[i][j]))
        if match:
            dictionary[str(leftPane[i][j])[70:100]] = str(match.group(1))

In [6]:
data_dict = {}
for i in range(len(rightPane)):
    for j in range(len(rightPane[i])):
        if str(rightPane[i][j])[71:101] in dictionary:
            html = str(rightPane[i][j])
            key = dictionary[str(rightPane[i][j])[71:101]]
            company_name = dictionary[str(leftPane[i][j])[70:100]]
            data_dict[key] = {'company_name' : company_name, 'percentage' : '', 'location' : '','url' : '', 'industry' : '', '#layoff' : '', 'stage' : '', 'date' : ''}
            url_pattern = r'<span class="url">(.*?)</span>'
            url_match = re.search(url_pattern, html)
            if url_match:
                url = url_match.group(1)
                data_dict[key]['url'] = url
            # Extract series
            common_pattern = r'<div class="flex-auto truncate-pre" title="(.*?)">'
            common_match = re.findall(common_pattern, html)
            if common_match:
                common = [common_match]
                data_dict[key]['location'] = common_match[0]
                if len(common_match)==4:
                    data_dict[key]['country'] = common_match[3]
                    data_dict[key]['stage'] = common_match[2]
                    data_dict[key]['industry'] = common_match[1]
                elif len(common_match)==3:
                    data_dict[key]['country'] = common_match[2]
                else:
                    data_dict[key]['country'] = common_match[4]
                    data_dict[key]['industry'] = common_match[2]
            pattern = r'<div class="flex-auto truncate line-height-4 right-align tabular-nums" style="padding:6px">(.*?)</div>'
            match = re.findall(pattern, str(rightPane[i][j]))
            if match:
                if len(match) == 3:
                    data_dict[key]['#layoff'] = match[0]
                    data_dict[key]['percentage'] = match[1]
                    data_dict[key]['raised'] = match[2]
                elif len(match) == 1:
                    data_dict[key]['raised'] = match[0]
                elif len(match) == 2:
                    if '%' in match[0]:
                        data_dict[key]['percentage'] = match[0]
                        data_dict[key]['raised'] = match[1]
                    elif '%' in match[1]:
                        data_dict[key]['#layoff'] = match[0]
                        data_dict[key]['percentage'] = match[1]
                    else:
                        data_dict[key]['#layoff'] = match[0]
                        data_dict[key]['raised'] = match[1]
            # Extract date
            date_pattern = r'<div class="truncate css-10jy3hn">(.*?)</div></div></div>'
            date_match = re.search(date_pattern, html)
            if date_match:
                date = date_match.group(1)
                data_dict[key]['date'] = date 
print(len(data_dict))


1513


In [7]:
# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data_dict, orient='index')

In [8]:
df.to_csv('../data/layoffs_fyi_dataset.csv')